In [1]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


## Step 1: Estrazione Frame

In [2]:
video_folder = "/content/drive/MyDrive/Magistrale/Tesi/Video Scenari/Bozza5"
video_sunny_folder = video_folder + "/Sunny"
video_gt_folder = video_folder + "/GT/Sunny"

frame_folder = "/content/drive/MyDrive/Magistrale/Tesi/SynthOffRoadReducted"
frame_sunny_folder = frame_folder + "/Images/Sunny"
frame_gt_folder = frame_folder + "/GT/Sunny"

Per primo step creiamo una grande unica cartella in cui mettere tutti i frame e poi dividiamo in train valid e test

In [3]:
import cv2
import os

def unpack_video(video_path, output_dir, perc_frames = -1):
    # Open the video file
    cap = cv2.VideoCapture(video_path)
    frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    print(f"Number of frames: {frame_count}")

    if not cap.isOpened():
        print("Error: Could not open video file")
        return

    # Create the output directory if it doesn't exist
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    # Read the video frame by frame
    frame_num = 0
    while True:
        ret, frame = cap.read()
        if not ret:
            break

        # Save the frame as an image
        image_path = os.path.join(output_dir, f"{frame_num:06d}.png")
        cv2.imwrite(image_path, frame)

        frame_num += 1

        if perc_frames > 0 and frame_num/frame_count >= perc_frames:
            print(f"Stopped at frame {frame_num} of the video")
            break

    cap.release()

def invert_grayscale_video(input_path, output_path):
    # Check if the input path exists
    if not os.path.exists(input_path):
        print(f"Error: The input file '{input_path}' does not exist.")
        return

    # Open the video file
    cap = cv2.VideoCapture(input_path)
    if not cap.isOpened():
        print(f"Error: Could not open video file '{input_path}'")
        return

    # Get video properties
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps = cap.get(cv2.CAP_PROP_FPS)
    codec = cv2.VideoWriter_fourcc(*'mp4v')  # Codec for mp4 format

    # Create a VideoWriter object to save the output video
    out = cv2.VideoWriter(output_path, codec, fps, (width, height), isColor=False)

    while True:
        # Read each frame
        ret, frame = cap.read()
        if not ret:
            break

        # Convert to grayscale
        gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

        # Perform gray-level inversion
        inverted_frame = 255 - gray_frame

        # Write the inverted frame to the output video
        out.write(inverted_frame)

    # Release resources
    cap.release()
    out.release()
    print(f"Inverted video saved as '{output_path}'")

In [4]:
!rm -r {frame_folder}

In [5]:
# Take all the *.avi files in the video folder (there are more levels of folder) with glob

import glob

video_files = glob.glob(video_folder + "/**/*.avi", recursive=True)
video_files

['/content/drive/MyDrive/Magistrale/Tesi/Video Scenari/Bozza5/GT/FL0.avi',
 '/content/drive/MyDrive/Magistrale/Tesi/Video Scenari/Bozza5/GT/FL1_2.avi',
 '/content/drive/MyDrive/Magistrale/Tesi/Video Scenari/Bozza5/GT/FL3.avi',
 '/content/drive/MyDrive/Magistrale/Tesi/Video Scenari/Bozza5/GT/FL4.avi',
 '/content/drive/MyDrive/Magistrale/Tesi/Video Scenari/Bozza5/GT/WL_inv0.avi',
 '/content/drive/MyDrive/Magistrale/Tesi/Video Scenari/Bozza5/GT/WL_inv1.avi',
 '/content/drive/MyDrive/Magistrale/Tesi/Video Scenari/Bozza5/GT/WL_inv2.avi',
 '/content/drive/MyDrive/Magistrale/Tesi/Video Scenari/Bozza5/GT/WL_inv3.avi',
 '/content/drive/MyDrive/Magistrale/Tesi/Video Scenari/Bozza5/GT/WL_inv4.avi',
 '/content/drive/MyDrive/Magistrale/Tesi/Video Scenari/Bozza5/Sunny/FL0.avi',
 '/content/drive/MyDrive/Magistrale/Tesi/Video Scenari/Bozza5/Sunny/FL1_2.avi',
 '/content/drive/MyDrive/Magistrale/Tesi/Video Scenari/Bozza5/Sunny/FL3.avi',
 '/content/drive/MyDrive/Magistrale/Tesi/Video Scenari/Bozza5/Sunny

In [6]:
video_files_clean = []

for video_path in video_files:
    if "Rainy" in video_path:
        print("Skipping ", video_path)
        continue

    if "WL" in video_path and not "inv" in video_path and "GT" in video_path:
        print("Skipping ", video_path)
        continue

    video_files_clean.append(video_path)

video_files = video_files_clean
video_files

Skipping  /content/drive/MyDrive/Magistrale/Tesi/Video Scenari/Bozza5/Rainy_GT/FL0.avi
Skipping  /content/drive/MyDrive/Magistrale/Tesi/Video Scenari/Bozza5/Rainy_GT/FL1_2.avi
Skipping  /content/drive/MyDrive/Magistrale/Tesi/Video Scenari/Bozza5/Rainy_GT/FL3.avi
Skipping  /content/drive/MyDrive/Magistrale/Tesi/Video Scenari/Bozza5/Rainy_GT/FL4.avi
Skipping  /content/drive/MyDrive/Magistrale/Tesi/Video Scenari/Bozza5/Rainy_GT/WL0.avi
Skipping  /content/drive/MyDrive/Magistrale/Tesi/Video Scenari/Bozza5/Rainy_GT/WL1.avi
Skipping  /content/drive/MyDrive/Magistrale/Tesi/Video Scenari/Bozza5/Rainy_GT/WL2.avi
Skipping  /content/drive/MyDrive/Magistrale/Tesi/Video Scenari/Bozza5/Rainy_GT/WL3.avi
Skipping  /content/drive/MyDrive/Magistrale/Tesi/Video Scenari/Bozza5/Rainy_GT/WL4.avi
Skipping  /content/drive/MyDrive/Magistrale/Tesi/Video Scenari/Bozza5/Rainy/FL0.avi
Skipping  /content/drive/MyDrive/Magistrale/Tesi/Video Scenari/Bozza5/Rainy/FL1_2.avi
Skipping  /content/drive/MyDrive/Magistrale/T

['/content/drive/MyDrive/Magistrale/Tesi/Video Scenari/Bozza5/GT/FL0.avi',
 '/content/drive/MyDrive/Magistrale/Tesi/Video Scenari/Bozza5/GT/FL1_2.avi',
 '/content/drive/MyDrive/Magistrale/Tesi/Video Scenari/Bozza5/GT/FL3.avi',
 '/content/drive/MyDrive/Magistrale/Tesi/Video Scenari/Bozza5/GT/FL4.avi',
 '/content/drive/MyDrive/Magistrale/Tesi/Video Scenari/Bozza5/GT/WL_inv0.avi',
 '/content/drive/MyDrive/Magistrale/Tesi/Video Scenari/Bozza5/GT/WL_inv1.avi',
 '/content/drive/MyDrive/Magistrale/Tesi/Video Scenari/Bozza5/GT/WL_inv2.avi',
 '/content/drive/MyDrive/Magistrale/Tesi/Video Scenari/Bozza5/GT/WL_inv3.avi',
 '/content/drive/MyDrive/Magistrale/Tesi/Video Scenari/Bozza5/GT/WL_inv4.avi',
 '/content/drive/MyDrive/Magistrale/Tesi/Video Scenari/Bozza5/Sunny/FL0.avi',
 '/content/drive/MyDrive/Magistrale/Tesi/Video Scenari/Bozza5/Sunny/FL1_2.avi',
 '/content/drive/MyDrive/Magistrale/Tesi/Video Scenari/Bozza5/Sunny/FL3.avi',
 '/content/drive/MyDrive/Magistrale/Tesi/Video Scenari/Bozza5/Sunny

In [7]:
for video_path in video_files:
    # Create the destination path
    video_name = video_path.split("/")[-1].split(".")[0]
    if "GT" in video_path:
        frame_path = frame_gt_folder + "/" + video_name
    else:
        frame_path = frame_sunny_folder + "/" + video_name

    frame_path = frame_path.split(".")[0]

    print(video_path + " -> " + frame_path)

    if not os.path.exists(frame_path):
        unpack_video(video_path, frame_path, perc_frames = 0.25)
    else:
        print("Skipping " + video_path + " Folder already exists")

/content/drive/MyDrive/Magistrale/Tesi/Video Scenari/Bozza5/GT/FL0.avi -> /content/drive/MyDrive/Magistrale/Tesi/SynthOffRoadReducted/GT/Sunny/FL0
Number of frames: 6041
Stopped at frame 1511 of the video
/content/drive/MyDrive/Magistrale/Tesi/Video Scenari/Bozza5/GT/FL1_2.avi -> /content/drive/MyDrive/Magistrale/Tesi/SynthOffRoadReducted/GT/Sunny/FL1_2
Number of frames: 8101
Stopped at frame 2026 of the video
/content/drive/MyDrive/Magistrale/Tesi/Video Scenari/Bozza5/GT/FL3.avi -> /content/drive/MyDrive/Magistrale/Tesi/SynthOffRoadReducted/GT/Sunny/FL3
Number of frames: 9652
Stopped at frame 2413 of the video
/content/drive/MyDrive/Magistrale/Tesi/Video Scenari/Bozza5/GT/FL4.avi -> /content/drive/MyDrive/Magistrale/Tesi/SynthOffRoadReducted/GT/Sunny/FL4
Number of frames: 5372
Stopped at frame 1343 of the video
/content/drive/MyDrive/Magistrale/Tesi/Video Scenari/Bozza5/GT/WL_inv0.avi -> /content/drive/MyDrive/Magistrale/Tesi/SynthOffRoadReducted/GT/Sunny/WL_inv0
Number of frames: 385

In [9]:
# For each folder, check if the elements inside the folder are the same in the folder under GT with same name

color_folders = glob.glob(frame_sunny_folder + "/*")
print(color_folders)

for color_folder in color_folders:
    gt_folder = color_folder.replace("Images", "GT")
    print(color_folder + " -> " + gt_folder)
    color_files = len(os.listdir(color_folder))
    gt_files = len(os.listdir(gt_folder))

    print(f"{color_folder}: {color_files} color files - {gt_files} gt files")


['/content/drive/MyDrive/Magistrale/Tesi/SynthOffRoadReducted/Images/Sunny/FL0', '/content/drive/MyDrive/Magistrale/Tesi/SynthOffRoadReducted/Images/Sunny/FL1_2', '/content/drive/MyDrive/Magistrale/Tesi/SynthOffRoadReducted/Images/Sunny/FL3', '/content/drive/MyDrive/Magistrale/Tesi/SynthOffRoadReducted/Images/Sunny/FL4', '/content/drive/MyDrive/Magistrale/Tesi/SynthOffRoadReducted/Images/Sunny/WL0', '/content/drive/MyDrive/Magistrale/Tesi/SynthOffRoadReducted/Images/Sunny/WL1', '/content/drive/MyDrive/Magistrale/Tesi/SynthOffRoadReducted/Images/Sunny/WL2', '/content/drive/MyDrive/Magistrale/Tesi/SynthOffRoadReducted/Images/Sunny/WL3', '/content/drive/MyDrive/Magistrale/Tesi/SynthOffRoadReducted/Images/Sunny/WL4']
/content/drive/MyDrive/Magistrale/Tesi/SynthOffRoadReducted/Images/Sunny/FL0 -> /content/drive/MyDrive/Magistrale/Tesi/SynthOffRoadReducted/GT/Sunny/FL0
/content/drive/MyDrive/Magistrale/Tesi/SynthOffRoadReducted/Images/Sunny/FL0: 1513 color files - 1511 gt files
/content/driv

In [10]:
# For each couple of folder color and gt take the one with few images and remove the images not available in the other folder (with greater id, so)

color_folders = glob.glob(frame_sunny_folder + "/*")
print(color_folders)

for color_folder in color_folders:
    gt_folder = color_folder.replace("Images", "GT")
    print(color_folder + " -> " + gt_folder)

    color_files = glob.glob(color_folder + "/*.png")
    gt_files = glob.glob(gt_folder + "/*.png")
    color_files.sort(reverse = True)
    gt_files.sort(reverse = True)
    print(f"{color_folder}: {len(color_files)} color files - {len(gt_files)} gt files")

    if len(color_files) > len(gt_files):
        for i in range(len(color_files) - len(gt_files)):
            os.remove(color_files[i])
            print("Removed " + color_files[i] + " from color")
    elif len(color_files) < len(gt_files):
        for i in range(len(gt_files) - len(color_files)):
            os.remove(gt_files[i])
            print("Removed " + gt_files[i] + " from GT")

['/content/drive/MyDrive/Magistrale/Tesi/SynthOffRoadReducted/Images/Sunny/FL0', '/content/drive/MyDrive/Magistrale/Tesi/SynthOffRoadReducted/Images/Sunny/FL1_2', '/content/drive/MyDrive/Magistrale/Tesi/SynthOffRoadReducted/Images/Sunny/FL3', '/content/drive/MyDrive/Magistrale/Tesi/SynthOffRoadReducted/Images/Sunny/FL4', '/content/drive/MyDrive/Magistrale/Tesi/SynthOffRoadReducted/Images/Sunny/WL0', '/content/drive/MyDrive/Magistrale/Tesi/SynthOffRoadReducted/Images/Sunny/WL1', '/content/drive/MyDrive/Magistrale/Tesi/SynthOffRoadReducted/Images/Sunny/WL2', '/content/drive/MyDrive/Magistrale/Tesi/SynthOffRoadReducted/Images/Sunny/WL3', '/content/drive/MyDrive/Magistrale/Tesi/SynthOffRoadReducted/Images/Sunny/WL4']
/content/drive/MyDrive/Magistrale/Tesi/SynthOffRoadReducted/Images/Sunny/FL0 -> /content/drive/MyDrive/Magistrale/Tesi/SynthOffRoadReducted/GT/Sunny/FL0
/content/drive/MyDrive/Magistrale/Tesi/SynthOffRoadReducted/Images/Sunny/FL0: 1513 color files - 1511 gt files
Removed /cont

In [11]:
# For each file in each images folder check that there is one with the same name in the GT folder

color_folders = glob.glob(frame_sunny_folder + "/*")
print(color_folders)

for color_folder in color_folders:
    gt_folder = color_folder.replace("Images", "GT")
    print(color_folder + " -> " + gt_folder)

    color_files = glob.glob(color_folder + "/*.png")

    for color_file in color_files:
        gt_file = color_file.replace("Images", "GT")
        if not os.path.exists(gt_file):
            print("GT file not found for " + color_file)

['/content/drive/MyDrive/Magistrale/Tesi/SynthOffRoadReducted/Images/Sunny/FL0', '/content/drive/MyDrive/Magistrale/Tesi/SynthOffRoadReducted/Images/Sunny/FL1_2', '/content/drive/MyDrive/Magistrale/Tesi/SynthOffRoadReducted/Images/Sunny/FL3', '/content/drive/MyDrive/Magistrale/Tesi/SynthOffRoadReducted/Images/Sunny/FL4', '/content/drive/MyDrive/Magistrale/Tesi/SynthOffRoadReducted/Images/Sunny/WL0', '/content/drive/MyDrive/Magistrale/Tesi/SynthOffRoadReducted/Images/Sunny/WL1', '/content/drive/MyDrive/Magistrale/Tesi/SynthOffRoadReducted/Images/Sunny/WL2', '/content/drive/MyDrive/Magistrale/Tesi/SynthOffRoadReducted/Images/Sunny/WL3', '/content/drive/MyDrive/Magistrale/Tesi/SynthOffRoadReducted/Images/Sunny/WL4']
/content/drive/MyDrive/Magistrale/Tesi/SynthOffRoadReducted/Images/Sunny/FL0 -> /content/drive/MyDrive/Magistrale/Tesi/SynthOffRoadReducted/GT/Sunny/FL0
/content/drive/MyDrive/Magistrale/Tesi/SynthOffRoadReducted/Images/Sunny/FL1_2 -> /content/drive/MyDrive/Magistrale/Tesi/Syn

In [12]:
import glob
# Take the frame files under the "Images" folder
frame_files = glob.glob(frame_sunny_folder + "/**/*.png", recursive=True)
print("Number of frame extracted: " + str(len(frame_files)))

Number of frame extracted: 18253


In [11]:
"""train_per, val_per, test_per = 0.7, 0.2, 0.1"""

In [27]:
"""!rm -r /content/drive/MyDrive/Magistrale/Tesi/SynthOffRoad/Test
!rm -r /content/drive/MyDrive/Magistrale/Tesi/SynthOffRoad/Train
!rm -r /content/drive/MyDrive/Magistrale/Tesi/SynthOffRoad/Val"""

In [26]:
"""# Create directories for train test and val and move files

import random
import shutil
from distutils.dir_util import copy_tree

random.seed(18)

# Shuffle the list of scenario folders
scenario_folders = glob.glob(frame_sunny_folder + "/**", recursive=False)
random.shuffle(scenario_folders)

train_scenarios = frame_files[:int(len(scenario_folders)*train_per)]
val_scenarios = frame_files[int(len(scenario_folders)*train_per):int(len(scenario_folders)*(train_per+val_per))]
test_scenarioss = frame_files[int(len(scenario_folders)*(train_per+val_per)):]

# Create the directories
train_dir = frame_folder + "/Train"
val_dir = frame_folder + "/Val"
test_dir = frame_folder + "/Test"

if not os.path.exists(train_dir):
    os.makedirs(train_dir)
    os.makedirs(train_dir + "/images")
    os.makedirs(train_dir + "/GT")
if not os.path.exists(val_dir):
    os.makedirs(val_dir)
    os.makedirs(val_dir + "/images")
    os.makedirs(val_dir + "/GT")
if not os.path.exists(test_dir):
    os.makedirs(test_dir)
    os.makedirs(test_dir + "/images")
    os.makedirs(test_dir + "/GT")

for scenario_folder in train_scenarios:
    dest_images = scenario_folder.replace("Sunny", "Train/images")
    dest_gt = scenario_folder.replace("Sunny", "Train/GT")
    copy_tree(scenario_folder, dest_images)
    copy_tree(scenario_folder.replace("Sunny", "GT"), dest_gt)
    print(scenario_folder + " -> " + dest_images)

for scenario_folder in val_scenarios:
    dest_images = scenario_folder.replace("Sunny", "Val/images")
    dest_gt = scenario_folder.replace("Sunny", "Val/GT")
    copy_tree(scenario_folder, dest_images)
    copy_tree(scenario_folder.replace("Sunny", "GT"), dest_gt)
    print(scenario_folder + " -> " + dest_images)

for scenario_folder in val_scenarios:
    dest_images = scenario_folder.replace("Sunny", "Test/images")
    dest_gt = scenario_folder.replace("Sunny", "Test/GT")
    copy_tree(scenario_folder, dest_images)
    copy_tree(scenario_folder.replace("Sunny", "GT"), dest_gt)
    print(scenario_folder + " -> " + dest_images)
"""

KeyboardInterrupt: 

In [16]:
"""# Create directories for train test and val and move files

import random
import shutil
random.seed(18)

# Shuffle the list of files
random.shuffle(frame_files)

# Split the list of files using the percentages
train_files = frame_files[:int(len(frame_files)*train_per)]
val_files = frame_files[int(len(frame_files)*train_per):int(len(frame_files)*(train_per+val_per))]
test_files = frame_files[int(len(frame_files)*(train_per+val_per)):]

# Create the directories
train_dir = frame_folder + "/Train"
val_dir = frame_folder + "/Val"
test_dir = frame_folder + "/Test"

if not os.path.exists(train_dir):
    os.makedirs(train_dir)
    os.makedirs(train_dir + "/images")
    os.makedirs(train_dir + "/GT")
if not os.path.exists(val_dir):
    os.makedirs(val_dir)
    os.makedirs(val_dir + "/images")
    os.makedirs(val_dir + "/GT")
if not os.path.exists(test_dir):
    os.makedirs(test_dir)
    os.makedirs(test_dir + "/images")
    os.makedirs(test_dir + "/GT")

# Move the files
for color_image in train_files:
    # Take the name of the parent folder
    image_dest_path = "/".join(color_image.replace("Sunny", "Train/images").split("/")[:-1])
    gt_dest_path = "/".join(color_image.replace("Sunny", "Train/GT").split("/")[:-1])

    if not os.path.exists(image_dest_path):
        os.makedirs(image_dest_path)
    if not os.path.exists(gt_dest_path):
        os.makedirs(gt_dest_path)

    # If there is also the corresponding GT file, move it
    try:
        shutil.copy(color_image.replace("Sunny", "GT"), gt_dest_path)
        shutil.copy(color_image, image_dest_path)
    except FileNotFoundError:
        print("GT file not available for " + color_image + " Skipping...")

for color_image in val_files:
    # Take the name of the parent folder
    image_dest_path = "/".join(color_image.replace("Sunny", "Val/images").split("/")[:-1])
    gt_dest_path = "/".join(color_image.replace("Sunny", "Val/GT").split("/")[:-1])

    if not os.path.exists(image_dest_path):
        os.makedirs(image_dest_path)
    if not os.path.exists(gt_dest_path):
        os.makedirs(gt_dest_path)

    # If there is also the corresponding GT file, move it
    try:
        shutil.copy(color_image.replace("Sunny", "GT"), gt_dest_path)
        shutil.copy(color_image, image_dest_path)
    except FileNotFoundError:
        print("GT file not available for " + color_image + " Skipping...")

for color_image in test_files:
    # Take the name of the parent folder
    image_dest_path = "/".join(color_image.replace("Sunny", "Test/images").split("/")[:-1])
    gt_dest_path = "/".join(color_image.replace("Sunny", "Test/GT").split("/")[:-1])

    print(image_dest_path)
    if not os.path.exists(image_dest_path):
        os.makedirs(image_dest_path)
    if not os.path.exists(gt_dest_path):
        os.makedirs(gt_dest_path)

    # If there is also the corresponding GT file, move it
    try:
        shutil.copy(color_image.replace("Sunny", "GT"), gt_dest_path)
        shutil.copy(color_image, image_dest_path)
    except FileNotFoundError:
        print("GT file not available for " + color_image + " Skipping...")"""

GT file not available for /content/drive/MyDrive/Magistrale/Tesi/SynthOffRoad/Sunny/WL4/014214.png Skipping...
GT file not available for /content/drive/MyDrive/Magistrale/Tesi/SynthOffRoad/Sunny/WL1/005501.png Skipping...
GT file not available for /content/drive/MyDrive/Magistrale/Tesi/SynthOffRoad/Sunny/WL2/000510.png Skipping...
GT file not available for /content/drive/MyDrive/Magistrale/Tesi/SynthOffRoad/Sunny/WL1/007666.png Skipping...
GT file not available for /content/drive/MyDrive/Magistrale/Tesi/SynthOffRoad/Sunny/WL1/010674.png Skipping...
GT file not available for /content/drive/MyDrive/Magistrale/Tesi/SynthOffRoad/Sunny/WL1/010470.png Skipping...
GT file not available for /content/drive/MyDrive/Magistrale/Tesi/SynthOffRoad/Sunny/WL4/003696.png Skipping...
GT file not available for /content/drive/MyDrive/Magistrale/Tesi/SynthOffRoad/Sunny/WL4/008861.png Skipping...
GT file not available for /content/drive/MyDrive/Magistrale/Tesi/SynthOffRoad/Sunny/WL0/003642.png Skipping...
G

KeyboardInterrupt: 

In [ ]:
"""# Count elements in each folder

train_images = glob.glob(train_dir + "/images/**/*.png", recursive=True)
train_gt = glob.glob(train_dir + "/GT/**/*.png", recursive=True)
val_images = glob.glob(val_dir + "/images/**/*.png", recursive=True)
val_gt = glob.glob(val_dir + "/GT/**/*.png", recursive=True)
test_images = glob.glob(test_dir + "/images/**/*.png", recursive=True)
test_gt = glob.glob(test_dir + "/GT/**/*.png", recursive=True)

print(f"Train: {len(train_images)} images - {len(train_gt)} gt")
print(f"Val: {len(val_images)} images - {len(val_gt)} gt")
print(f"Test: {len(test_images)} images - {len(test_gt)} gt")"""

In [ ]:
import glob
import os

# Remove all .DS_Store files

ds_store_files = glob.glob(frame_folder + "/**/.DS_Store", recursive=True)
print("Num of .DS_Store files found: " + str(len(ds_store_files)))

for ds_store_file in ds_store_files:
    os.remove(ds_store_file)

In [13]:
# For each folder under GT/Sunny and Images/Sunny rename it as Sunny_<original_name>

import shutil

sunny_folders = glob.glob(frame_sunny_folder + "/**", recursive=False)
print(sunny_folders)

print("DOING WITH IMAGES")

for sunny_folder in sunny_folders:
    print(sunny_folder)

    # Move the folder in the parent directory with the new name
    new_name = sunny_folder.split("/")[-1]
    new_name = "Sunny_" + new_name

    # Rename the folder
    new_name_folder = sunny_folder.replace(sunny_folder.split("/")[-1], new_name)
    os.rename(sunny_folder, new_name_folder)
    print(sunny_folder + " -> " + new_name_folder)

    print("Moving " + new_name_folder + " to " + os.path.dirname(os.path.dirname(new_name_folder)))
    shutil.move(new_name_folder, os.path.dirname(os.path.dirname(new_name_folder)))

print("DOING WITH GT")

sunny_folders = glob.glob(frame_gt_folder + "/**", recursive=False)
print(sunny_folders)

for sunny_folder in sunny_folders:
    print(sunny_folder)

    # Move the folder in the parent directory with the new name
    new_name = sunny_folder.split("/")[-1]
    new_name = "Sunny_" + new_name

    # Rename the folder
    new_name_folder = sunny_folder.replace(sunny_folder.split("/")[-1], new_name)
    os.rename(sunny_folder, new_name_folder)
    print(sunny_folder + " -> " + new_name_folder)

    print("Moving " + new_name_folder + " to " + os.path.dirname(os.path.dirname(new_name_folder)))
    shutil.move(new_name_folder, os.path.dirname(os.path.dirname(new_name_folder)))



['/content/drive/MyDrive/Magistrale/Tesi/SynthOffRoadReducted/Images/Sunny/FL0', '/content/drive/MyDrive/Magistrale/Tesi/SynthOffRoadReducted/Images/Sunny/FL1_2', '/content/drive/MyDrive/Magistrale/Tesi/SynthOffRoadReducted/Images/Sunny/FL3', '/content/drive/MyDrive/Magistrale/Tesi/SynthOffRoadReducted/Images/Sunny/FL4', '/content/drive/MyDrive/Magistrale/Tesi/SynthOffRoadReducted/Images/Sunny/WL0', '/content/drive/MyDrive/Magistrale/Tesi/SynthOffRoadReducted/Images/Sunny/WL1', '/content/drive/MyDrive/Magistrale/Tesi/SynthOffRoadReducted/Images/Sunny/WL2', '/content/drive/MyDrive/Magistrale/Tesi/SynthOffRoadReducted/Images/Sunny/WL3', '/content/drive/MyDrive/Magistrale/Tesi/SynthOffRoadReducted/Images/Sunny/WL4']
DOING WITH IMAGES
/content/drive/MyDrive/Magistrale/Tesi/SynthOffRoadReducted/Images/Sunny/FL0
/content/drive/MyDrive/Magistrale/Tesi/SynthOffRoadReducted/Images/Sunny/FL0 -> /content/drive/MyDrive/Magistrale/Tesi/SynthOffRoadReducted/Images/Sunny/Sunny_FL0
Moving /content/dri

In [65]:
!rm -r /content/drive/MyDrive/Magistrale/Tesi/SynthOffRoadReducted/GT/Sunny
!rm -r /content/drive/MyDrive/Magistrale/Tesi/SynthOffRoadReducted/Images/Sunny

In [14]:
!mkdir /content/drive/MyDrive/Magistrale/Tesi/SynthOffRoadReducted/GT/Train
!mkdir /content/drive/MyDrive/Magistrale/Tesi/SynthOffRoadReducted/GT/Val
!mkdir /content/drive/MyDrive/Magistrale/Tesi/SynthOffRoadReducted/GT/Test

!mkdir /content/drive/MyDrive/Magistrale/Tesi/SynthOffRoadReducted/Images/Train
!mkdir /content/drive/MyDrive/Magistrale/Tesi/SynthOffRoadReducted/Images/Val
!mkdir /content/drive/MyDrive/Magistrale/Tesi/SynthOffRoadReducted/Images/Test

In [15]:
!mv /content/drive/MyDrive/Magistrale/Tesi/SynthOffRoadReducted/Images/Sunny_FL3 /content/drive/MyDrive/Magistrale/Tesi/SynthOffRoadReducted/Images/Val
!mv /content/drive/MyDrive/Magistrale/Tesi/SynthOffRoadReducted/Images/Sunny_WL4 /content/drive/MyDrive/Magistrale/Tesi/SynthOffRoadReducted/Images/Val

!mv /content/drive/MyDrive/Magistrale/Tesi/SynthOffRoadReducted/Images/Sunny_FL0 /content/drive/MyDrive/Magistrale/Tesi/SynthOffRoadReducted/Images/Test
!mv /content/drive/MyDrive/Magistrale/Tesi/SynthOffRoadReducted/Images/Sunny_WL3 /content/drive/MyDrive/Magistrale/Tesi/SynthOffRoadReducted/Images/Test

!mv /content/drive/MyDrive/Magistrale/Tesi/SynthOffRoadReducted/Images/Sunny_FL1_2 /content/drive/MyDrive/Magistrale/Tesi/SynthOffRoadReducted/Images/Train
!mv /content/drive/MyDrive/Magistrale/Tesi/SynthOffRoadReducted/Images/Sunny_FL4 /content/drive/MyDrive/Magistrale/Tesi/SynthOffRoadReducted/Images/Train
!mv /content/drive/MyDrive/Magistrale/Tesi/SynthOffRoadReducted/Images/Sunny_WL0 /content/drive/MyDrive/Magistrale/Tesi/SynthOffRoadReducted/Images/Train
!mv /content/drive/MyDrive/Magistrale/Tesi/SynthOffRoadReducted/Images/Sunny_WL1 /content/drive/MyDrive/Magistrale/Tesi/SynthOffRoadReducted/Images/Train
!mv /content/drive/MyDrive/Magistrale/Tesi/SynthOffRoadReducted/Images/Sunny_WL2 /content/drive/MyDrive/Magistrale/Tesi/SynthOffRoadReducted/Images/Train

!mv /content/drive/MyDrive/Magistrale/Tesi/SynthOffRoadReducted/GT/Sunny_FL3 /content/drive/MyDrive/Magistrale/Tesi/SynthOffRoadReducted/GT/Val
!mv /content/drive/MyDrive/Magistrale/Tesi/SynthOffRoadReducted/GT/Sunny_WL4 /content/drive/MyDrive/Magistrale/Tesi/SynthOffRoadReducted/GT/Val

!mv /content/drive/MyDrive/Magistrale/Tesi/SynthOffRoadReducted/GT/Sunny_FL0 /content/drive/MyDrive/Magistrale/Tesi/SynthOffRoadReducted/GT/Test
!mv /content/drive/MyDrive/Magistrale/Tesi/SynthOffRoadReducted/GT/Sunny_WL3 /content/drive/MyDrive/Magistrale/Tesi/SynthOffRoadReducted/GT/Test

!mv /content/drive/MyDrive/Magistrale/Tesi/SynthOffRoadReducted/GT/Sunny_FL1_2 /content/drive/MyDrive/Magistrale/Tesi/SynthOffRoadReducted/GT/Train
!mv /content/drive/MyDrive/Magistrale/Tesi/SynthOffRoadReducted/GT/Sunny_FL4 /content/drive/MyDrive/Magistrale/Tesi/SynthOffRoadReducted/GT/Train
!mv /content/drive/MyDrive/Magistrale/Tesi/SynthOffRoadReducted/GT/Sunny_WL0 /content/drive/MyDrive/Magistrale/Tesi/SynthOffRoadReducted/GT/Train
!mv /content/drive/MyDrive/Magistrale/Tesi/SynthOffRoadReducted/GT/Sunny_WL1 /content/drive/MyDrive/Magistrale/Tesi/SynthOffRoadReducted/GT/Train
!mv /content/drive/MyDrive/Magistrale/Tesi/SynthOffRoadReducted/GT/Sunny_WL2 /content/drive/MyDrive/Magistrale/Tesi/SynthOffRoadReducted/GT/Train

In [16]:
!zip -rq /content/drive/MyDrive/Magistrale/Tesi/SynthOffRoadReducted.zip /content/drive/MyDrive/Magistrale/Tesi/SynthOffRoadReducted